<a href="https://colab.research.google.com/github/DIPANJAN001/TalkJulia_with_Randy/blob/main/SolarPowerManagementForSir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [104]:
class MyEnv3(gym.Env):
      
      def __init__(self,seed=42):
        np.random.seed(seed)
        self.action_space = Discrete(4)
        self.observation_space=MultiDiscrete([150,130,50])
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        self.state,self.reward, self.done, self.info= None, None, None,None
        self.counter=0
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
      def _render(self):
        txt="state:"+str(self.state)+"reward"+str(self.reward)+"info:"+str(self.info)
        print(txt)

      def _states(self):
        observation_space=[]
        return observation_space
      def _reset(self):
        self.battery_cap=30
        self.state,self.reward,self.done,self.info=[30,0,0],0,False,{}
        return self.state

      def _step(self,action,load,pv):
        grid_export_=0
        grid_import_=0
        done=False
        #load=np.random.randint(100,140)
        #pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        reward=0
        data=[]
        if action== 0:
          grid_export_= pv-load-self.max_battery_charge # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= pv-load-self.max_battery_charge # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=load-pv-self.max_battery_discharge
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=load-pv-self.max_battery_discharge
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        self.counter=self.counter+1
        if self.counter==500:
          done=True
        '''if(self.battery_cap<7.0):
              done=True'''
        #data.append([battery_state,load,pv,action,reward])
        #state.append(load)
        #state.append(pv)
        return reward,self.battery_cap,info,done

In [105]:
env=MyEnv3()

In [106]:
import time # Necessary to evaluate frugality
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import deque

In [107]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        #self.memory = deque(maxlen=2000)
        self.memory=[]
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=3,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(4,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self): #method that trains NN with experiences sampled from memory
        #minibatch = random.sample(self.memory,batch_size)
        minibatch = agent.memory[0:len(agent.memory)]
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict((next_state,))[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
            

In [108]:
agent = Agent(state_size=3,action_size=4)
done = False
state_size = 3
action_size =4
batch_size = 32

In [109]:

  score = 0
  #load=np.random.randint(100,140)
  #pv=np.random.randint(80,130)
  state=env._reset()

  #state_=[]
  #state_.append(state[0])
  next_state=[]
  while not done:
    next_state=[]
    #state_.append(state[0])
    #load=np.random.randint(100,140)
    #pv=np.random.randint(80,130)
    #state_.append(load)
    #state_.append(pv)
    #print(state_)
    #state1=state_
    #action = env.action_space.sample()'''
    print(state)
    action=agent.act(state)
    #print(action)
    reward,observation2,info,done = env._step(action,state[1],state[2])

    load=np.random.randint(100,140)
    pv=np.random.randint(80,130)
    next_state.append(observation2)
    next_state.append(load)
    next_state.append(pv)
    #print(next_state)
    agent.remember(state,action,reward,next_state,done)
    score+=round(reward)
    state=next_state
    '''next_state.pop(0)
    next_state.pop(0)
    next_state.pop(0)'''
    #print(observation2)
    print(info)
    
    #state_[0]=observation2
    #agent.store_transition()
  print('episode:{} Profit:{}'.format(e,score))

[30, 0, 0]
{'grid_export': 0, 'grid_import': -15}
[15, 128, 94]
{'grid_export': 0, 'grid_import': 19}
[0, 120, 118]
{'grid_export': -12, 'grid_import': 0}
[0, 122, 90]
{'grid_export': -42, 'grid_import': 0}
[0, 135, 119]
{'grid_export': -26, 'grid_import': 0}
[0, 121, 81]
{'grid_export': 0, 'grid_import': 25}
[0, 129, 117]
{'grid_export': 0, 'grid_import': -3}
[0, 120, 112]
{'grid_export': -18, 'grid_import': 0}
[0, 121, 123]
{'grid_export': 0, 'grid_import': -17}
[2, 126, 121]
{'grid_export': -15, 'grid_import': 0}
[0, 115, 94]
{'grid_export': -31, 'grid_import': 0}
[0, 102, 116]
{'grid_export': 4, 'grid_import': 0}
[10, 120, 88]
{'grid_export': -42, 'grid_import': 0}
[0, 103, 104]
{'grid_export': 0, 'grid_import': -16}
[1, 108, 105]
{'grid_export': 0, 'grid_import': -12}
[0, 119, 107]
{'grid_export': -22, 'grid_import': 0}
[0, 106, 123]
{'grid_export': 0, 'grid_import': -32}
[17, 134, 93]
{'grid_export': -51, 'grid_import': 0}
[0, 139, 83]
{'grid_export': -66, 'grid_import': 0}
[0, 1

In [110]:
agent.memory

[([30, 0, 0], 2, 150, [15, 128, 94], False),
 ([15, 128, 94], 3, -190, [0, 120, 118], False),
 ([0, 120, 118], 0, -96, [0, 122, 90], False),
 ([0, 122, 90], 1, -336, [0, 135, 119], False),
 ([0, 135, 119], 0, -208, [0, 121, 81], False),
 ([0, 121, 81], 2, -250, [0, 129, 117], False),
 ([0, 129, 117], 3, 30, [0, 120, 112], False),
 ([0, 120, 112], 0, -144, [0, 121, 123], False),
 ([0, 121, 123], 3, 170, [2, 126, 121], False),
 ([2, 126, 121], 0, -120, [0, 115, 94], False),
 ([0, 115, 94], 0, -248, [0, 102, 116], False),
 ([0, 102, 116], 1, 32, [10, 120, 88], False),
 ([10, 120, 88], 1, -336, [0, 103, 104], False),
 ([0, 103, 104], 3, 160, [1, 108, 105], False),
 ([1, 108, 105], 3, 120, [0, 119, 107], False),
 ([0, 119, 107], 1, -176, [0, 106, 123], False),
 ([0, 106, 123], 3, 320, [17, 134, 93], False),
 ([17, 134, 93], 0, -408, [0, 139, 83], False),
 ([0, 139, 83], 0, -528, [0, 105, 121], False),
 ([0, 105, 121], 0, 48, [10, 128, 97], False),
 ([10, 128, 97], 0, -328, [0, 133, 89], Fal

In [111]:
minibatch = agent.memory[0:len(agent.memory)-1]
for state,action,reward,next_state,done in minibatch:
            
    if not done: #boolean 
          target = reward + agent.gamma*np.amax(agent.model.predict((next_state,))[0])
    else:
          target = reward
          target_f = agent.model.predict(np.array(state))
          target_f[0][action] = target
          agent.model.fit(state,target_f,epochs=1,verbose=0)

1/1 [==============================] - 0s 33ms/step


In [113]:
print(np.argmax(agent.model.predict(([0, 134, 112],))[0]))

1/1 [==============================] - 0s 31ms/step
3
